In [ ]:
# Install dependencies

%pip install torch==2.6.0 torchvision==0.21.0 tqdm==4.67.1 typer==0.15.1 pyarrow==18.1.0 setuptools==75.1.0 protobuf==4.25.3 ultralytics==8.3.107 albumentations==2.0.5

In [ ]:
# Imports

from ultralytics import YOLO
import glob
import os
import shutil
import torch

In [ ]:
# Utils

def get_device() -> str:
    try:
        return 0 if torch.cuda.is_available() else "cpu"
    except Exception as e:
        print(f"Error setting device: {e}")

def remove_models():
    pt_files = glob.glob("*.pt")
    print("Files to be removed:", pt_files)

    for file in pt_files:
        os.remove(file)

In [ ]:
# Run this only once to set up the dataset cache the first time.

# Downloads the VisDrone dataset and runs the first epoch to create the label.cache file with the original labels.
# The custom dataset is later created on the fly by mapping the original labels to the new ones.
# Without this step, the custom dataset would not be created correctly.

model = YOLO("yolo12n.pt")
results = model.train(
    data="VisDrone.yaml",
    device=get_device(),
    batch=16,
    workers=8,
    imgsz=640,
    epochs=1,
    exist_ok=True,
    save=False,
)

# Remove the saved data
if results.save_dir.exists():
    print(f"Removing {results.save_dir}")
    shutil.rmtree(results.save_dir)
            
del model

# Remove the dummy model
remove_models()